## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

from IPython.display import HTML

# import sys
# sys.path.append('../../')

from openbb_terminal.reports import widget_helpers as widgets
from openbb_terminal.sdk import openbb
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal.core.config.paths import REPOSITORY_DIRECTORY

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

# Detect if prediction capabilities are present. If they are not, disable prediction in the rest of the script
# so that the report can still be generated without prediction results.
# predictions = True
# try:
#     openbb.stocks.pred.models
# except Exception as e:
#     predictions = False

# TODO Fix predictions virtual path on api refactored

predictions = False

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "VTI"
report_name = "Equity Report for VTI"

In [ ]:
ticker = symbol

In [ ]:
if "." in ticker:
    import sys

    sys.exit(0)

In [ ]:
ticker_data = openbb.stocks.load(
    symbol=ticker, start_date=datetime.datetime.now() - datetime.timedelta(days=4 * 30)
)
ticker_data = openbb.stocks.process_candle(data=ticker_data)

author = "OpenBB"
report_title = f"ETF RESEARCH REPORT ON {ticker.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
info = openbb.etf.summary(name=ticker)
info

## Data

In [ ]:
data = openbb.stocks.load(ticker)

fig, (candles, volume) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.etf.candle(
    symbol=ticker,
    data=data,
    use_matplotlib=True,
    external_axes=[candles, volume],
)

candles.set_xticklabels("")
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
candle_chart = f.getvalue().decode("utf-8")

In [ ]:
import pandas as pd

weights = openbb.etf.weights(ticker)
weights_df = pd.DataFrame.from_dict([weights]).T
weights_df.columns = ["Percentage"]

In [ ]:
fig, ax = plt.subplots(dpi=150)
ax.pie(
    weights_df[weights_df["Percentage"] > 10]["Percentage"],
    labels=weights_df[weights_df["Percentage"] > 10].index,
    autopct="%1.1f%%",
    shadow=True,
    startangle=90,
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
weights_chart = f.getvalue().decode("utf-8")

In [ ]:
holdings = openbb.etf.holdings(ticker)
holdings

## Render the report template to a file

In [ ]:
body = ""

img = (
    str(REPOSITORY_DIRECTORY)
    + "/openbb_terminal/reports/templates/OpenBB_reports_logo.png"
)
floppy_disk_img = (
    str(REPOSITORY_DIRECTORY) + "/openbb_terminal/reports/templates/floppy-disc.png"
)
body += widgets.header(
    img,
    floppy_disk_img,
    author,
    report_date,
    report_time,
    report_timezone,
    f"<b>ETF RESEARCH REPORT:</b> {ticker.upper()}",
)

body += widgets.tablinks(["SUMMARY"])

htmlcode = widgets.h(3, "Summary")
htmlcode += widgets.p(info)
htmlcode += widgets.row([candle_chart])

htmlcode += widgets.p("The ETF has the following weights distribution:")

htmlcode += widgets.row([weights_df.to_html()])

htmlcode += widgets.p("And has funds in the following assets:")
htmlcode += widgets.row([holdings.to_html()])

body += widgets.add_tab("SUMMARY", htmlcode)

body += widgets.tab_clickable_and_save_evt()

report = widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)